In [1]:
#Installs
!pip install transformers
!pip install transformers[sentencepiece] sacrebleu
!pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.0/104.0 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 11.4 MB/s eta 0:00:00


In [6]:
#Imports
import pandas as pd
import numpy as np
import tensorflow as tf
import csv
import nltk
import torch
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast
from nltk.translate.bleu_score import corpus_bleu
from sentence_transformers import SentenceTransformer, util
from sacrebleu.metrics import CHRF
from nltk.translate.meteor_score import meteor_score
from nltk.lm import KneserNeyInterpolated
from nltk.lm.preprocessing import padded_everygram_pipeline
from nltk.util import ngrams

In [4]:
#Loading the model
pipe = pipeline("translation", model="facebook/mbart-large-50-many-to-many-mmt")
model= MBartForConditionalGeneration.from_pretrained("facebook/mbart-large-50-many-to-many-mmt").to('cuda')
tokenizer = MBart50TokenizerFast.from_pretrained("facebook/mbart-large-50-one-to-many-mmt")
translator = pipeline('translation', model=model, tokenizer=tokenizer, src_lang="hi_IN", tgt_lang='en_XX',max_length=400)

config.json:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.44G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/261 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/529 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/528 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/717 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [7]:
#Load Dataset
df=pd.read_csv('./Bhagwad_Gita.csv')
hindi_sentences=df['HinMeaning'].values
english_sentences=df['EngMeaning'].values

In [8]:
df.head()

,ID,Chapter,Verse,Shloka,Transliteration,HinMeaning,EngMeaning,WordMeaning
0,BG1.1,1,1,धृतराष्ट्र उवाच |\nधर्मक्षेत्रे कुरुक्षेत्रे स...,dhṛtarāṣṭra uvāca .\ndharmakṣetre kurukṣetre s...,।।1.1।।धृतराष्ट्र ने कहा -- हे संजय ! धर्मभूमि...,1.1 Dhritarashtra said What did my people and...,1.1 धर्मक्षेत्रे on the holy plain? कुरुक्षेत्...
1,BG1.2,1,2,सञ्जय उवाच |\nदृष्ट्वा तु पाण्डवानीकं व्यूढं द...,sañjaya uvāca .\ndṛṣṭvā tu pāṇḍavānīkaṃ vyūḍha...,।।1.2।।संजय ने कहा -- पाण्डव-सैन्य की व्यूह रच...,1.2. Sanjaya said Having seen the army of the...,1.2 दृष्ट्वा having seen? तु indeed? पाण्डवानी...
2,BG1.3,1,3,पश्यैतां पाण्डुपुत्राणामाचार्य महतीं चमूम् |\n...,paśyaitāṃ pāṇḍuputrāṇāmācārya mahatīṃ camūm .\...,।।1.3।।हे आचार्य ! आपके बुद्धिमान शिष्य द्रुपद...,"1.3. ""Behold, O Teacher! this mighty army of t...",1.3 पश्य behold? एताम् this? पाण्डुपुत्राणाम् ...
3,BG1.4,1,4,अत्र शूरा महेष्वासा भीमार्जुनसमा युधि |\nयुयुध...,atra śūrā maheṣvāsā bhīmārjunasamā yudhi .\nyu...,।।1.4।।इस सेना में महान् धनुर्धारी शूर योद्धा ...,"1.4. Here are heroes, mighty archers, eal in b...",1.4 अत्र here? शूराः heroes? महेष्वासाः mighty...
4,BG1.5,1,5,धृष्टकेतुश्चेकितानः काशिराजश्च वीर्यवान् |\nपु...,dhṛṣṭaketuścekitānaḥ kāśirājaśca vīryavān .\np...,"।।1.5।।धृष्टकेतु, चेकितान, बलवान काशिराज, पुर...","1.5. ""Dhrishtaketu, chekitana and the valiant ...",1.5 धृष्टकेतुः Dhrishtaketu? चेकितानः Chekitan...


In [14]:
Shlokas=df['Shloka'].values
hindi_sentences=df['HinMeaning'].values
english_sentences=df['EngMeaning'].values

In [15]:
translatedHinEng = []
for index, (hin, eng) in enumerate(zip(hindi_sentences, english_sentences)):
    print(index+1)
    print("Hindi:", hin)
    prediction = translator(hin)
    translation = prediction[0]['translation_text']
    print("Translated: ", translation)
    print("Original: ", eng)
    print("------------------------------")
    translatedHinEng.append(translation)

1
Hindi: ।।1.1।।धृतराष्ट्र ने कहा -- हे संजय ! धर्मभूमि कुरुक्षेत्र में एकत्र हुए युद्ध के इच्छुक (युयुत्सव:) मेरे और पाण्डु के पुत्रों ने क्या किया?
Translated:  Dhritarashtra said, “Hey Sanjay, what did my sons and Pandu, who wanted to fight in Dharma Bhoomi Kurukshetra, do? ”
Original:  1.1 Dhritarashtra said  What did my people and the sons of Pandu do when they had assembled
together eager for battle on the holy plain of Kurukshetra, O Sanjaya.
------------------------------
2
Hindi: ।।1.2।।संजय ने कहा -- पाण्डव-सैन्य की व्यूह रचना देखकर राजा दुर्योधन ने आचार्य द्रोण के पास जाकर ये वचन कहे।
Translated:  Sanjay said: When King Duryodhana saw the vision of the Pandava Army, he went to Acharya Drona and said these words.
Original:  1.2. Sanjaya said  Having seen the army of the Pandavas drawn up in battle-array,
King Duryodhana then approached his teacher (Drona) and spoke these words.
------------------------------
3
Hindi: ।।1.3।।हे आचार्य ! आपके बुद्धिमान शिष्य द्रुपदपुत्र (धृष्टद

In [16]:
#Saving the results
comaparision = pd.DataFrame({'Original': english_sentences, 'Translated': translatedHinEng})
comaparision.to_csv('/content/BG-MBART50-Translated-Hin_Eng.csv', index=False)

###Calculating the various accuracy scores
---
```
*   BLEU: 43.06
*   CHRF: 36.1312
*   STS: 0.6896
*   METEOR: 0.3238
*   ORIGINAL PERPLEXITY: 19.5507
*   TRANSLATED PERPLEXITY: 19.6691
```

In [17]:
#BLEU Score
references = [[english] for english in english_sentences]  # Create reference list of lists
candidates = translatedHinEng  # Model translations
bleu_score = corpus_bleu(references, candidates)
print(f"BLEU score: {bleu_score:.4f}")

BLEU score: 0.4306


In [18]:
# STS Score
# Load the pre-trained model
model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')
model = model.to('cuda' if torch.cuda.is_available() else 'cpu')

# Encode the original Hindi sentences and the translated sentences
embeddings1 = model.encode(english_sentences, convert_to_tensor=True, device=model.device)
embeddings2 = model.encode(translatedHinEng, convert_to_tensor=True, device=model.device)

# Compute cosine similarities between the embeddings
cosine_scores = util.cos_sim(embeddings1, embeddings2)

# Calculate the average STS score
sts_score = cosine_scores.diag().mean().item()
print(f"STS score: {sts_score:.4f}")

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.12k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/645 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

STS score: 0.6896


In [19]:
# CHRF
# Initialize CHRF metric
chrf = CHRF(word_order=1)

# Calculate CHRF score
chrf_score = chrf.corpus_score(translatedHinEng, [[english] for english in english_sentences]).score
print(f"CHRF score: {chrf_score:.4f}")

CHRF score: 36.1312


In [20]:
# METEOR Score
# Download required NLTK data (if not already downloaded)
nltk.download('punkt')  # Download the 'punkt' resource for sentence tokenization
nltk.download('wordnet')

# Tokenize the sentences if they're not already tokenized
english_sentences_tokenized = [nltk.word_tokenize(sent) for sent in english_sentences]
translatedHinEng_tokenized = [nltk.word_tokenize(sent) for sent in translatedHinEng]

# Calculate METEOR score
from nltk.translate.meteor_score import meteor_score # Import the meteor_score function
meteor_scores = [meteor_score([ref], hyp) for ref, hyp in zip(english_sentences_tokenized, translatedHinEng_tokenized)]
average_meteor_score = sum(meteor_scores) / len(meteor_scores)

print(f"METEOR score: {average_meteor_score:.4f}")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


METEOR score: 0.3238


In [21]:
# PERPLEXITY
def train_model(sentences, n=2):
    tokenized = [['<s>'] + list(map(str.lower, nltk.word_tokenize(sent))) + ['</s>'] for sent in sentences]
    train_data, padded_vocab = padded_everygram_pipeline(n, tokenized)
    model = KneserNeyInterpolated(n)
    model.fit(train_data, padded_vocab)
    return model

def calculate_perplexity(model, sentences, n=2):
    perplexities = []
    for sent in sentences:
        tokens = ['<s>'] + list(map(str.lower, nltk.word_tokenize(sent))) + ['</s>']
        test_data = list(ngrams(tokens, n))
        try:
            pp = model.perplexity(test_data)
            perplexities.append(pp)
        except ValueError:
            pass  # Skip sentences that cause errors
    return np.mean(perplexities) if perplexities else float('inf')

# Train models
original_model = train_model(english_sentences)
translation_model = train_model(translatedHinEng)

# Calculate perplexities
original_perplexity = calculate_perplexity(original_model, english_sentences)
translated_perplexity = calculate_perplexity(translation_model, translatedHinEng)

print(f"Original Perplexity: {original_perplexity:.4f}")
print(f"Translated Perplexity: {translated_perplexity:.4f}")

Hindi Perplexity: 19.5507
Translated English Perplexity: 19.6691
